# Binary search trees


## Definition

A binary search tree is a tree where
- each node stores a value
- each node is either a leaf or has one or two children
- the left child has a value smaller than the node, and the right larger

![bst](resources/bst.png)
Topics to worry about

- Finding an element
- Inserting and element
- Reversing the tree
- Creating the tree

In [9]:
class Node:
    """
    Node class
    """
    def __init__(self, value):
        self.value = value
        self.child_left = None
        self.child_right = None

    def get_left(self):
        return self.child_left

    def get_right(self):
        return self.child_right
      
    def __repr__(self):
        r = "none" if self.child_right is None else self.child_right.value
        l = "none" if self.child_left is None else self.child_left.value
        return f"(key = {self.value}, ({l}, {r}))"
        

In [2]:
four = Node(4)
four.child_left = Node(5)
four

(key = 4, (5, none))

## Complexity

Search and insertion are proportional to he height of the tree. In a worst case scenario the tree is a continuous line, so the complexity is $O(n)$.

## Insertion

Query the current root. If it's equal to the key we want inserted, ignore (WARNING: assumption). If it's smaller than the key we want inserted, we query the children nodes.

In [7]:
class BST:
    """
    Binary search tree class
    """
    def __init__(self):
        self.leader = None
            
    def search(self, key, root = None):
        """
        Go either left or right until the key/leaf  is found
        """
        if key == self.leader.value:
            return True
        elif root is None: # maybe raise KeyError?
            return False
        elif root.value == key:
            return True        
        elif root.value < key:
            print("")
            # key is larger than the root, should be on the right side
            return self.search(root.get_right(), key)
        elif root.value > key:
            # key is smaller than the root, should be on the left side
            return self.search(root.get_left(), key)
    
    def insert(self, node):
        print("="*60)
        if self.leader is None:
            print(f"setting top node to {node.value}")
            self.leader = node
        else:
            self.insert_rec(self.leader, node.value)
            
    def insert_rec(self, root, key):
        """
        Go either left or right until you either find the key or you find a leaf
        """ 
        if root is None:
            print(f"inserting {key}")
            root = Node(key)
        elif root.value < key:
            print(f"key {key} is larger than  root {root.value}")
            # key is larger, go right
            self.insert_rec(root.get_right(), key)   
        elif root.value > key:
            # key is smaller, go left
            print(f"key {key} is smaller than  root {root.value}")
            self.insert_rec(root.get_left(), key)
        # Question: what about duplicates?
        elif root.value == key:
            print("key equal")
    
    def in_order(self):
        """
        returns the nodes in the 
        """
        ...
        # To do, use yield statements
        

In [8]:
b = BST()
b.insert(Node(4))
b.insert(Node(5))
b.insert(Node(6))
b

setting top node to 4
key 5 is larger than  root 4
inserting 5
key 6 is larger than  root 4
inserting 6


In [5]:
b.search(6)

False

# Further reading

[Geeks for Geeks](http://www.geeksforgeeks.org/binary-search-tree-set-1-search-and-insertion/)